<a href="https://colab.research.google.com/github/wizard339/DLSchool/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22report_metr_exp_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
import numpy as np
import re
import zipfile
from collections import namedtuple

In [70]:
data = pd.read_excel('/content/Для анализа.xlsx', index_col='2')
data.head()

,Дата получения документа,Дата возврата документа,Номер извещения,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры)
2,,,,,,,,,,,
1,2024-01-09,2024-01-09,NaN,ИТКС.733421.005,Основание,"КО, Авилов",8,Марусина,NaN,NaN,NaN
2,2024-01-09,2024-01-09,NaN,ИТКС.402132.032 ГЧ,Блок чувствительных элементов БЧЭ-23,"КО, Авилов",4,Марусина,NaN,NaN,NaN
3,2024-01-09,2024-01-09,NaN,ИТКС.402132.032 СБ,Блок чувствительных элементов БЧЭ-23,"КО, Авилов",8,Марусина,NaN,NaN,NaN
4,2024-01-10,2024-01-11,NaN,ИТКС.738-23,КМИВ.402113.007 ГВК-6-1,"ОРСДВ, Селицкая",1,Иванов,NaN,NaN,NaN
5,2024-01-10,2024-01-11,NaN,КМИВ.713352.004 Д2,КМИВ.402113.007 ГВК-6-1,"ОРСДВ, Селицкая",9,Иванов,NaN,"5, 21",21


# 1 Обработка пропусков в данных

In [71]:
# заполняем пропуски в столбце 'Отдел, разработчик' значениями предыдущей строки
data['Отдел, разработчик'] = data['Отдел, разработчик'].fillna(method='ffill')

<ipython-input-71-bd7cc067c892>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Отдел, разработчик'] = data['Отдел, разработчик'].fillna(method='ffill')


In [72]:
data['Дата получения документа'] = data['Дата получения документа'].fillna(method='ffill')
data['Дата возврата документа'] = data['Дата возврата документа'].fillna(data['Дата получения документа'])

<ipython-input-72-1d69fbcec0d6>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Дата получения документа'] = data['Дата получения документа'].fillna(method='ffill')


In [73]:
data['Есть повторно выданные замечания'] = np.where(data['Повторные замечания (шифры)'].isna(), 0, 1)
data[' Замечания (шифры)'] = data[' Замечания (шифры)'].fillna(0)
data['Повторные замечания (шифры)'] = data['Повторные замечания (шифры)'].fillna(0)

# 2 Создание дополнительных столбцов

In [74]:
# формируем список нерабочих дней за 2024 год согласно производственному календарю

# список всех СБ и ВС в 2024 году
sat_sun = pd.bdate_range(start='2024-01-01', end='2024-12-31', freq='C', weekmask='Sat Sun')
# рабочие СБ и ВС
work_weekends = {'2024-04-27', '2024-11-02', '2024-12-28'}
# праздничные дни
holidays = {
    '2024-01-01',
    '2024-01-02',
    '2024-01-03',
    '2024-01-04',
    '2024-01-05',
    '2024-01-08',
    '2024-02-23',
    '2024-03-08',
    '2024-04-29',
    '2024-04-30',
    '2024-05-01',
    '2024-05-09',
    '2024-05-10',
    '2024-06-12',
    '2024-11-04',
    '2024-12-30',
    '2024-12-31',
}
# формируем множество всех СБ и ВС в формате строки "2024-12-29"
not_workdays = set()

for d in sat_sun:
    not_workdays.add(str(d)[:10])

# формируем итоговое множество как объединение множества всех СБ и ВС
# с множество праздничных дней, а затем исключаем из него рабочие СБ и ВС,
# далее для удобства преобразуем в список
all_not_workdays = list(not_workdays.union(holidays) - work_weekends)

# ну и на всякий случай проверка, 366 дней в 2024 году, согласно производственному
# календарю рабочих дней 248, а значит нерабочих: 366 - 248 = 118 дней
assert(len(all_not_workdays)==118)

In [75]:
data['Срок проверки'] = [
    len(pd.bdate_range(
        start=data['Дата получения документа'].iloc[i],
        end=data['Дата возврата документа'].iloc[i],
        freq='C',
        weekmask='Mon Tue Wed Thu Fri Sat Sun',
        holidays=all_not_workdays,
        )) for i in range(data['Дата возврата документа'].shape[0])
    ]

In [76]:
data['Разработчик'] = ''

In [77]:
# разделяем значения из столбца 'Отдел, разработчик' на 2 значения
for i in range(data['Отдел, разработчик'].shape[0]):
    data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
    for s in data['Отдел, разработчик'].iloc[i]:
        s = s.strip()
        data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1].strip()
    data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][0]

Выходные данные были обрезаны до нескольких последних строк (5000).
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][0]
<ipython-input-77-63d391b8659b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][0]
<ipython-input-77-63d391b8659b>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermedi

In [78]:
# исправляем типичные ошибки в названиях отделов + актуализируем названия
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРМД', 'ОРМС'],'ОРМ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРЛГ', 'ОРИС')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРСТВГ', 'ОРС ТВГ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРСДВ', 'ОРС ДВ'], 'ОРС ДНГ и ВОГ')

data['Разработчик'] = data['Разработчик'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Мальцева', 'Мацюк'], 'Мацюк (Мальцева)')

In [79]:
# добавляем к фамилии разработчика суффикс с названием его отдела
for i in range(data['Разработчик'].shape[0]):
    data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]

Выходные данные были обрезаны до нескольких последних строк (5000).
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]
<ipython-input-79-15cfcd48afa6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]
<ipython-input-79-15cfcd48afa6>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the

# 3 Обработка столбца с замечаниями

In [80]:
# для удобства дальнейшей обработки создаем список шифров
for i in range(data[' Замечания (шифры)'].shape[0]):
    a = [s.strip() for s in re.split('[,.]', str(data[' Замечания (шифры)'].iloc[i]))]
    data[' Замечания (шифры)'].iloc[i] = a

for i in range(data['Повторные замечания (шифры)'].shape[0]):
    a = [s.strip() for s in re.split('[,.]', str(data['Повторные замечания (шифры)'].iloc[i]))]
    data['Повторные замечания (шифры)'].iloc[i] = a

Выходные данные были обрезаны до нескольких последних строк (5000).
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['Повторные замечания (шифры)'].iloc[i] = a
<ipython-input-80-cf212622269c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Повторные замечания (шифры)'].iloc[i] = a
<ipython-input-80-cf212622269c>:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave 

In [81]:
data.sample(10)

,Дата получения документа,Дата возврата документа,Номер извещения,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры),Есть повторно выданные замечания,Срок проверки,Разработчик
2,,,,,,,,,,,,,,
1885,2024-07-01,2024-07-01,ИТКС.496 2/2-24,ИТКС.496 2/2-24,ИТКС.402113.016 ГВК-6-1-1,ОРС ДНГ и ВОГ,1,Иванов,присвоение литеры,[0],[0],0,1,Мищенко_ОРС ДНГ и ВОГ
1191,2024-05-28,2024-05-29,NaN,ИТКС.301539.010СБ,Соединитель,ОРС ТВГ,1,Авилов,NaN,[0],[0],0,2,Долгополов_ОРС ТВГ
1275,2024-05-31,2024-05-31,NaN,ИТКС.745512.037,Экран,КО,2,Авилов,NaN,[20],[0],0,1,Плигин_КО
2706,2024-10-17,2024-10-17,NaN,ИТКС.687253.123 СБ,Плата многослойная,КО,2,Марусина,NaN,[0],[0],0,1,Кудрявцев_КО
364,2024-02-20,2024-02-20,NaN,ИТКС.758753.022,Плата,ОРС ТВГ,1,Марусина,NaN,[0],[0],0,1,Ахметзянов_ОРС ТВГ
2292,2024-08-15,2024-08-15,NaN,ИТКС.671121.019 Д2,Трансформатор,ОРИС,9,Сидорова,NaN,[0],[0],0,1,Арефьев_ОРИС
800,2024-04-16,2024-04-17,NaN,ИТКС.714574.002,Вал,ОРС ТВГ,2,Авилов,NaN,[16],[0],0,2,Козарь_ОРС ТВГ
17,2024-01-12,2024-01-12,NaN,ИТКС.468119.002 ГЧ,Блок электроники,КО,2,Марусина,NaN,[0],[0],0,1,Авилов_КО
1180,2024-05-28,2024-05-29,NaN,ИТКС.301319.010СБ,Кронштейн,ОРС ТВГ,1,Авилов,NaN,"[10, 17, 29]",[0],0,2,Долгополов_ОРС ТВГ


In [36]:
# функция для подсчета количества ошибок по отделам

def count_codes(data: pd.DataFrame, department: str) -> dict:
    # словарь для хранения количества замечаний по каждому шифру
    codes = dict(zip(np.arange(0, 31), np.zeros(30)))

    # цикл по строкам датафрейма  с помощью указателя i
    for i in range(data.shape[0]):

        # если строка относится к нужному отделу
        if data['Отдел, разработчик'].iloc[i] == department:
            # берем значение из столбца с замечаниями

            for c in data[' Замечания (шифры)'].iloc[i]:
                print(i)
                # добавляем к счетчику шифров
                codes[int(c)] += 1

            for c in data['Повторные замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров за исключением 0 шифра
                if int(c) != 0:
                    codes[int(c)] += 1

    return codes

In [82]:
def count_codes(data: pd.DataFrame, department: str) -> dict:
    # словарь для хранения количества замечаний по каждому шифру
    codes = dict(zip(np.arange(0, 31), np.zeros(30)))

    repeated_codes = dict(zip(np.arange(0, 31), np.zeros(30)))

    # цикл по строкам датафрейма  с помощью указателя i
    for i in range(data.shape[0]):
        # если строка относится к нужному отделу
        if data['Отдел, разработчик'].iloc[i] == department:
            # берем значение из столбца с замечаниями

            for c in data[' Замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров
                print(i)
                codes[int(c)] += 1
            for c in data['Повторные замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров за исключением 0 шифра
                if int(c) != 0:
                    codes[int(c)] += 1
                    repeated_codes[int(c)] += 1

    return codes, repeated_codes

In [83]:
# функция для подсчета количества ошибок по разработчику

def count_codes_by_person(data: pd.DataFrame, person: str) -> dict:
    # словарь для хранения количества замечаний по каждому шифру
    codes = dict(zip(np.arange(0, 31), np.zeros(30)))

    # цикл по строкам датафрейма  с помощью указателя i
    for i in range(data.shape[0]):

        # если строка относится к нужному разработчику
        if data['Разработчик'].iloc[i] == person:

            # берем значение из столбца с замечаниями
            for c in data[' Замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров
                codes[int(c)] += 1

            for c in data['Повторные замечания (шифры)'].iloc[i]:
                # добавляем к счетчику шифров за исключением 0 шифра
                if int(c) != 0:
                    codes[int(c)] += 1

    return codes

# 4 Формирование отчетов

In [84]:
def make_main_report(data: pd.DataFrame) -> pd.DataFrame:
    report = pd.DataFrame(columns=['Проверено документов, шт',
                                    'Проверено листов А4, шт',
                                    'Всего замечаний, шт',
                                    'Всего замечаний без 17 кода, шт',
                                    'Доля документов с замечаниями, %',
                                    'Количество замечаний на 100 листов А4, шт',
                                    'Количество замечаний на 100 листов А4 без 17 кода, шт',
                                    'Доля документов с повторно выданными замечаниями, %'])
    report['Проверено документов, шт'] = data.groupby(['Отдел, разработчик']).count()['Дата получения документа']
    report['Проверено листов А4, шт'] = data[['Отдел, разработчик', 'Количество форматов А4']].groupby(['Отдел, разработчик']).sum()

    # в цикле создаем вспомогательные переменные для подсчета замечаний и показателей
    for name in report.index.to_list():
        # создаем словарь для каждого отдела с количеством замечаний
        codes = count_codes(data, name)[0]
        # переменная для хранения количества замечаний без учета шифра '17'
        total_without_17 = 0
        # переменная для хранения количества замечаний с шифром '17'
        count_17 = codes[17]
        # переменная для хранения количества документов без замечаний
        without_mistakes = codes[0]
        for c in codes.keys():
            if c not in (0, 17):
                total_without_17 += codes[c]
        # переменная для хранения общего числа замечаний
        total_count = total_without_17 + count_17
        # переменная для хранения количества документов с повторно выданными замечаниями
        docs_with_repeated_codes = np.sum(data['Есть повторно выданные замечания'].loc[data['Отдел, разработчик']==name])

        report.loc[name, 'Всего замечаний, шт'] = total_count
        report.loc[name, 'Всего замечаний без 17 кода, шт'] = total_without_17
        report.loc[name, 'Доля документов с замечаниями, %'] = np.round(((1 - (without_mistakes / report.loc[name, 'Проверено документов, шт'])) * 100), 1)
        report.loc[name, 'Количество замечаний на 100 листов А4, шт'] = np.round(((total_count / report.loc[name, 'Проверено листов А4, шт']) * 100), 1)
        report.loc[name, 'Количество замечаний на 100 листов А4 без 17 кода, шт'] = np.round(((total_without_17 / report.loc[name, 'Проверено листов А4, шт']) * 100), 1)
        report.loc[name, 'Доля документов с повторно выданными замечаниями, %'] = np.round((docs_with_repeated_codes / (report.loc[name, 'Проверено документов, шт'] - without_mistakes) * 100), 1)

    # создаем переменные для хранения сводных показателей по предприятию
    all_docs = np.sum(report['Проверено документов, шт'])
    all_pages = np.sum(report['Проверено листов А4, шт'])
    all_mistakes = np.sum(report['Всего замечаний, шт'])
    all_mistakes_without_17 = np.sum(report['Всего замечаний без 17 кода, шт'])
    all_docs_with_mistakes = np.sum(report['Проверено документов, шт'] * (report['Доля документов с замечаниями, %'] / 100))
    all_docs_with_repeated_codes = np.sum(data['Есть повторно выданные замечания'])

    # добавляем в датафрейм строку со сводными показателями
    report.loc['Общая статистика по предприятию'] = [all_docs, all_pages, all_mistakes, all_mistakes_without_17, np.round((all_docs_with_mistakes / all_docs)*100, 1), np.round((all_mistakes / all_pages)*100, 1), np.round((all_mistakes_without_17 / all_pages)*100, 1), np.round((all_docs_with_repeated_codes / all_docs_with_mistakes)*100, 1)]

    return report

In [85]:
def make_codes_report(data: pd.DataFrame) -> pd.DataFrame:
    Codes = namedtuple('Codes', ['department', 'codes'])
    counts_of_codes = [Codes(dp, count_codes(data, dp)[0]) for dp in data['Отдел, разработчик'].unique()]

    cols = list(data['Отдел, разработчик'].unique())
    cols.insert(0, 'Шифр')
    report = pd.DataFrame(columns=[cols])

    for nt in counts_of_codes:
        report[nt.department] = nt.codes.values()

    report['Шифр'] = report.index

    return report

In [86]:
def make_codes_by_person_report(data: pd.DataFrame) -> pd.DataFrame:
    Codes = namedtuple('Codes', ['person', 'codes'])
    counts_of_codes = [Codes(pers, count_codes_by_person(data, pers)) for pers in data['Разработчик'].unique()]

    cols = list(data['Разработчик'].unique())
    cols.insert(0, 'Шифр')
    report = pd.DataFrame(columns=[cols])

    for nt in counts_of_codes:
        report[nt.person] = nt.codes.values()

    report['Шифр'] = report.index
    report = report.T
    report = report.drop('Шифр', axis=0)

    report.insert(0, 'Отдел', [report.index.values[i][0].split('_')[1] for i in range(report.index.shape[0])])

    return report

In [87]:
def make_time_report(data: pd.DataFrame) -> pd.DataFrame:
    report = pd.DataFrame(data['Срок проверки'].value_counts())
    return report

In [88]:
def make_report_by_doctype(data: pd.DataFrame) -> pd.DataFrame:
    litera_otrabotka = data.loc[data['Проект, тема, литера/ примечания']=='отработка на лит.']
    litera_assignment = data.loc[data['Проект, тема, литера/ примечания']=='присвоение литеры']
    tec_condition_extension = data.loc[data['Проект, тема, литера/ примечания']=='продление ТУ']
    new_and_correction = data.loc[data['Проект, тема, литера/ примечания'].isna()]
    correction = new_and_correction.loc[new_and_correction['Номер извещения'].notna()]
    new_docs = new_and_correction.loc[new_and_correction['Номер извещения'].isna()]

    cols = ['Подвыборка', 'Количество документов, шт', 'Количество листов А4']
    report = pd.DataFrame(columns=[cols])

    report.loc[len(report.index)] = ['Отработка на литеру', len(litera_otrabotka['Количество форматов А4']), litera_otrabotka['Количество форматов А4'].sum()]
    report.loc[len(report.index)] = ['Присвоение литеры', len(litera_assignment['Количество форматов А4']), litera_assignment['Количество форматов А4'].sum()]
    report.loc[len(report.index)] = ['Продление ТУ', len(tec_condition_extension['Количество форматов А4']), tec_condition_extension['Количество форматов А4'].sum()]
    report.loc[len(report.index)] = ['По извещениям', len(correction['Количество форматов А4']), correction['Количество форматов А4'].sum()]
    report.loc[len(report.index)] = ['Новая документация', len(new_docs['Количество форматов А4']), new_docs['Количество форматов А4'].sum()]

    return report

# 5 Деление датасета на части

In [89]:
# разделим датасет на части по кварталам и полугодиям
data_1q = data[(data['Дата получения документа'] > '2024-01-01') & (data['Дата получения документа'] < '2024-04-01')]
data_2q = data[(data['Дата получения документа'] >= '2024-04-01') & (data['Дата получения документа'] < '2024-07-01')]
data_3q = data[(data['Дата получения документа'] >= '2024-07-01') & (data['Дата получения документа'] < '2024-10-01')]
data_4q = data[(data['Дата получения документа'] >= '2024-10-01') & (data['Дата получения документа'] < '2025-01-01')]
data_1_half_year = data[(data['Дата получения документа'] >= '2024-01-01') & (data['Дата получения документа'] < '2024-07-01')]
data_2_half_year = data[(data['Дата получения документа'] >= '2024-07-01') & (data['Дата получения документа'] < '2025-01-01')]
data_full = data[(data['Дата получения документа'] >= '2024-01-01') & (data['Дата получения документа'] < '2025-01-01')]
data_9m = data[(data['Дата получения документа'] >= '2024-01-01') & (data['Дата получения документа'] < '2024-10-01')]
# data_list = [data_1q, data_2q, data_3q, data_4q, data_1_half_year, data_2_half_year, data_full]
data_list = [data_3q, data_9m]

In [90]:
# создадим аттрибуты с именем датасета
data_1q.name = '1_квартал'
data_2q.name = '2_квартал'
data_3q.name = '3_квартал'
data_4q.name = '4_квартал'
data_1_half_year.name = '1_полугодие'
data_2_half_year.name = '2_полугодие'
data_full.name = 'Весь_год'
data_9m.name = '9_месяцев'

In [91]:
print(f'1 квартал: {data_1q.shape} \n2 квартал: {data_2q.shape} \n3 квартал: {data_3q.shape}  \n4 квартал: {data_4q.shape} \n1 полугодие: {data_1_half_year.shape}  \n2 полугодие: {data_2_half_year.shape} \nВесь год: {data_full.shape} \n9 месяцев: {data_9m.shape}')

1 квартал: (660, 14) 
2 квартал: (1217, 14) 
3 квартал: (697, 14)  
4 квартал: (136, 14) 
1 полугодие: (1877, 14)  
2 полугодие: (833, 14) 
Весь год: (2710, 14) 
9 месяцев: (2574, 14)


# 6 Сохранение отчетов

In [92]:
for d in data_list:
    with zipfile.ZipFile(f'{d.name}.zip', 'w') as zf:
        with zf.open(f'{d.name}_основной_отчет.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_main_report(d)
                df.to_excel(writer)

        with zf.open(f'{d.name}_основной_по_ошибкам.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_codes_report(d)
                df.to_excel(writer)

        with zf.open(f'{d.name}_ошибки_по_разработчику.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_codes_by_person_report(d)
                df.to_excel(writer)

        with zf.open(f'{d.name}_по_срокам.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_time_report(d)
                df.to_excel(writer)

        with zf.open(f'{d.name}_по_виду_документов.xlsx', 'w') as buffer:
            with pd.ExcelWriter(buffer) as writer:
                df = make_report_by_doctype(d)
                df.to_excel(writer)


19
20
21
22
23
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
66
67
68
69
70
71
82
85
86
87
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
202
203
221
222
223
224
244
245
246
249
250
251
252
253
254
255
256
259
260
261
262
263
264
265
266
267
268
271
272
291
292
293
294
302
303
304
305
310
311
312
313
314
315
316
317
318
319
320
321
323
323
324
325
328
329
330
331
332
333
334
335
344
345
346
347
348
349
350
351
352
353
354
355
359
360
369
370
386
387
388
389
390
391
392
393
399
400
401
402
407
408
409
410
411
412
413
417
418
419
420
421
422
423
424
429
431
432
433
434
435
436
437
438
441
448
449
450
451
473
518
519
531
532
535
536
537
538
539
540
547
548
549
553
554
555
556
557
558
559
560
561
582
583
590
591
592
599
600
601
602
607
608
609
612
613
618
619
620
621
622
623
624
631
632
633
656
657
658
661
662
663
664
665
666
667
668
677
678
679
680
681
686
689
690
691
692
5

<ipython-input-84-28e9bc735f40>:36: RuntimeWarning: invalid value encountered in scalar divide
  report.loc[name, 'Доля документов с повторно выданными замечаниями, %'] = np.round((docs_with_repeated_codes / (report.loc[name, 'Проверено документов, шт'] - without_mistakes) * 100), 1)



446
446
452
452
482
482
522
523
524
525
526
527
528
597
598
0
1
2
3
4
5
6
7
13
13
14
15
16
17
18
63
63
63
63
63
64
65
79
83
84
91
92
93
94
95
96
97
98
99
100
101
102
103
176
215
216
233
240
461
462
463
593
594
614
614
685
184
206
207
207
207
242
242
439
440
442
443
444
445
459
460
533
534
550
551
552
637
638
639
640
72
72
72
75
76
146
147
168
169
170
171
185
186
219
220
234
235
236
237
243
297
298
299
300
301
326
327
327
343
361
454
455
456
474
506
507
508
509
510
511
512
513
514
515
516
517
541
542
543
544
545
546
566
567
568
569
570
573
574
575
576
577
580
581
595
596
615
616
617
650
651
659
660
683
0
1
2
3
4
5
6
7
13
13
14
15
16
17
18
63
63
63
63
63
64
65
79
83
84
91
92
93
94
95
96
97
98
99
100
101
102
103
176
215
216
233
240
461
462
463
593
594
614
614
685
8
9
80
81
158
159
160
161
172
173
173
179
180
181
182
183
187
187
187
187
188
189
189
190
241
241
241
308
309
362
363
364
365
366
367
368
371
372
373
374
376
376
376
376
376
376
376
376
376
376
377
377
377
377
378
379
379
380
38

<ipython-input-86-bd3e9eababbd>:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  report = report.drop('Шифр', axis=0)


Выходные данные были обрезаны до нескольких последних строк (5000).
1943
1944
1945
1946
1947
1959
1962
1963
1964
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2079
2080
2098
2099
2100
2101
2121
2122
2123
2126
2127
2128
2129
2130
2131
2132
2133
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2148
2149
2168
2169
2170
2171
2179
2180
2181
2182
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2200
2200
2201
2202
2205
2206
2207
2208
2209
2210
2211
2212
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2236
2237
2246
2247
2263
2264
2265
2266
2267
2268
2269
2270
2276
2277
2278
2279
2284
2285
2286
2287
2288
2289
2290
2294
2295
2296
2297
2298
2299
2300
2301
2306
2308
2309
2310
2311
2312
2313
2314
2315
2318
2325
2326
2327
2328
2350
2395
2396
2408
2409
2412
2413
2414
2415
2416
2417
2424
2425
2426
2430
2431
2432
2433
2434
2435
2436
2437
2438
2459
2460
2467
2468
2469
2476
24

<ipython-input-86-bd3e9eababbd>:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  report = report.drop('Шифр', axis=0)
